In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/PHD/SemEval2023/SemEval2022-Task10/private_space

/content/gdrive/MyDrive/PHD/SemEval2023/SemEval2022-Task10/private_space


In [3]:
ls ../Results/TaskB/lexicon_*

../Results/TaskB/lexicon_pmi.json
../Results/TaskB/lexicon_shapley_augmented_v1.json
../Results/TaskB/lexicon_shapley_augmented_v2_shap.json
../Results/TaskB/lexicon_shapley_augmented_v2_sqrt.json
../Results/TaskB/lexicon_shapley_augmented_v3_gptj.json
../Results/TaskB/lexicon_shapley_augmented_v4_tweetbert.json
../Results/TaskB/lexicon_shapley.json


In [4]:
ls ../Results/TaskC/lexicon_*

../Results/TaskC/lexicon_pmi.json
../Results/TaskC/lexicon_shapley_augmented_v2_shap.json
../Results/TaskC/lexicon_shapley_augmented_v3_gptj.json
../Results/TaskC/lexicon_shapley_augmented_v4_tweetbert.json
../Results/TaskC/lexicon_shapley.json


In [5]:
# use github on google colab: https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

In [6]:
# !git config --global user.name "Pakawat Nakwijit"
# !git config --global user.email "p.nakwijit@qmul.ac.uk"

In [ ]:
# !git push origin main

In [7]:
import json

with open('../Results/TaskC/lexicon_pmi.json') as fin:
    lexicon_pmi = json.load(fin)

with open('../Results/TaskC/lexicon_shapley.json') as fin:
    lexicon_shapley = json.load(fin)

# with open('../Results/TaskC/lexicon_shapley_augmented_v1.json') as fin:
#     lexicon_shapley_augmented_v1 = json.load(fin)

# with open('../Results/TaskB/lexicon_shapley_augmented_v2_sqrt.json') as fin:
#     lexicon_shapley_augmented_v2_sqrt = json.load(fin)

with open('../Results/TaskC/lexicon_shapley_augmented_v2_shap.json') as fin:
    lexicon_shapley_augmented_v2_shap = json.load(fin)

with open('../Results/TaskC/lexicon_shapley_augmented_v3_gptj.json') as fin:
    lexicon_shapley_augmented_v3_gptj = json.load(fin)

with open('../Results/TaskC/lexicon_shapley_augmented_v4_tweetbert.json') as fin:
    lexicon_shapley_augmented_v4_tweetbert = json.load(fin)

    

In [8]:
# (lexicon_shapley_augmented_v1['1. threats, plans to harm and incitement']["sister"])

In [9]:
import warnings
warnings.filterwarnings("ignore")

## Load Data

In [10]:
import pandas as pd

target_column = "label_vector"
data = pd.read_csv("Data/starting_ki/train_all_tasks.csv")
data = data[data[target_column]!="none"]

label_values = [
      '1.1 threats of harm',
      '1.2 incitement and encouragement of harm',
      '2.1 descriptive attacks',
      '2.2 aggressive and emotive attacks',
      '2.3 dehumanising attacks & overt sexual objectification',
      '3.1 casual use of gendered slurs, profanities, and insults',
      '3.2 immutable gender differences and gender stereotypes',
      '3.3 backhanded gendered compliments',
      '3.4 condescending explanations or unwelcome advice',
      '4.1 supporting mistreatment of individual women',
      '4.2 supporting systemic discrimination against women as a group',
  ]

In [11]:
label_map = {
    0: '1.1 threats of harm',
    1: '1.2 incitement and encouragement of harm',
    2: '2.1 descriptive attacks',
    3: '2.2 aggressive and emotive attacks',
    4: '2.3 dehumanising attacks & overt sexual objectification',
    5: '3.1 casual use of gendered slurs, profanities, and insults',
    6: '3.2 immutable gender differences and gender stereotypes',
    7: '3.3 backhanded gendered compliments',
    8: '3.4 condescending explanations or unwelcome advice',
    9: '4.1 supporting mistreatment of individual women',
    10: '4.2 supporting systemic discrimination against women as a group',
    '1.1 threats of harm': 0,
    '1.2 incitement and encouragement of harm': 1,
    '2.1 descriptive attacks': 2,
    '2.2 aggressive and emotive attacks': 3,
    '2.3 dehumanising attacks & overt sexual objectification': 4,
    '3.1 casual use of gendered slurs, profanities, and insults': 5,
    '3.2 immutable gender differences and gender stereotypes': 6,
    '3.3 backhanded gendered compliments': 7,
    '3.4 condescending explanations or unwelcome advice': 8,
    '4.1 supporting mistreatment of individual women': 9,
    '4.2 supporting systemic discrimination against women as a group': 10,
}

In [12]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize the stopwords
stoplist = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
import string

def preprocess(sent, **kwargs):
  sent = sent.lower()
  words = nltk.word_tokenize(sent)

  # Remove Stop words
  words = [w for w in words if w not in stoplist]

  # Remove Punctuation
  words = [w for w in words if w not in string.punctuation]
  
  # Lematization
  if "lemma" in kwargs and kwargs["lemma"]:
    words = [lemmatizer.lemmatize(w) for w in words]
    
  return words

In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

## Logistic Regression with all words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def logistic_preprocess(sent):
  words = preprocess(sent, lemma=True)
  return words

def dummy_word_tokenize(sent):
  return sent

vectorizer = CountVectorizer(
    preprocessor=logistic_preprocess,
    tokenizer=dummy_word_tokenize
)

In [ ]:
X_train = vectorizer.fit_transform(train["text"].values)
X_test = vectorizer.transform(test["text"].values)

In [ ]:
y_train = [label_map[l] for l in train[target_column].values]
y_test = [label_map[l] for l in test[target_column].values]

In [ ]:
"#Vocab", len(vectorizer.get_feature_names_out())

('#Vocab', 7574)

In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn

model = LogisticRegression(random_state=42, penalty="l2")
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
print("F1: %.3f" % sklearn.metrics.f1_score(y_test, y_predict, average='macro'))
print("Acc: %.3f" % sklearn.metrics.accuracy_score(y_test, y_predict))

p, r, f1, sup = sklearn.metrics.precision_recall_fscore_support(y_test, y_predict, average="macro")
print("P: %.3f" % p)
print("R: %.3f" % r)

F1: 0.245
Acc: 0.376
P: 0.287
R: 0.241


In [ ]:
from tqdm import tqdm
import numpy as np


def cross_validation(data, fold=5):
  N = int(len(data)/fold)

  metrics = {
      "acc":[],
      "p":[],
      "r":[],
      "f1":[]
  }

  for k in tqdm(range(fold), total=fold):
    test = data.iloc[k*N:(k+1)*N]
    train = pd.concat([data.iloc[:k*N], data.iloc[(k+1)*N:]])
    
    vectorizer = CountVectorizer(
        preprocessor=logistic_preprocess,
        tokenizer=dummy_word_tokenize
    )

    X_train = vectorizer.fit_transform(train["text"].values)
    X_test = vectorizer.transform(test["text"].values)

    y_train = [label_map[l] for l in train[target_column].values]
    y_test = [label_map[l] for l in test[target_column].values]

    model = LogisticRegression(random_state=42, penalty="l2")
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)

    # f1 = metrics.f1_score(y_test, y_predict, average='macro')
    acc = sklearn.metrics.accuracy_score(y_test, y_predict)
    p, r, f1, sup = sklearn.metrics.precision_recall_fscore_support(y_test, y_predict, average="macro")

    metrics["acc"].append(acc)
    metrics["f1"].append(f1)
    metrics["p"].append(p)
    metrics["r"].append(r)
  
  print("")
  for m in metrics:
    print(f"{m}: {np.mean(metrics[m]):.3f} ± {np.std(metrics[m]):.3f}")
  return ;

cross_validation(data)

100%|██████████| 5/5 [00:24<00:00,  4.90s/it]


acc: 0.378 ± 0.007
p: 0.288 ± 0.053
r: 0.237 ± 0.016
f1: 0.240 ± 0.020


## Lexicon counting

In [15]:
from sklearn.metrics import f1_score
import sklearn

def predict(words, lexicons):
  cnt = {}
  for label in lexicons:
    cnt[label] = 0

  for w in words:
    for label in lexicons:
      if w in lexicons[label]:
        cnt[label] += lexicons[label][w]

  if sum(cnt.values())==0:
    return None
  
  return max(cnt.items(), key=lambda k: k[1])[0]


def run_eval(data, lexicons, **kwargs):
  texts = data["text"].values
  sents = [preprocess(t, **kwargs) for t in texts]
  labels = data[target_column].values

  y_pred = []
  y_test = []
  non = 0
  for words, label in zip(sents, labels):
    pred = predict(words, lexicons)
    if pred is None:
      non += 1
      continue

    y_pred.append(pred)
    y_test.append(label)

  print(f"F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
  p, r, f1, sup = sklearn.metrics.precision_recall_fscore_support(y_test, y_pred, average="macro")
  print(f"P {p:.3f}")
  print(f"R {r:.3f}")

  print(f"Skip: {non/len(sents):.3f}")
  return

In [16]:
# lexicon_pmi
# lexicon_shapley
# lexicon_shapley_augmented_v2_shap
# lexicon_shapley_augmented_v3_gptj

In [18]:
print("Evaluate PMI Lexicons")
run_eval(test, lexicon_pmi, lemma=True)

Evaluate PMI Lexicons
F1: 0.088
P 0.095
R 0.095
Skip: 0.379


In [ ]:
print("Evaluate Shapley Lexicons")
run_eval(test, lexicon_shapley, lemma=False)

Evaluate Shapley Lexicons
F1: 0.146
P 0.177
R 0.180
Skip: 0.009


In [ ]:
# print("Evaluate Augmented Shapley Lexicons")
# run_eval(test, lexicon_shapley_augmented_v1, lemma=False)

In [ ]:
# print("Evaluate Augmented Shapley Lexicons")
# run_eval(test, lexicon_shapley_augmented_v2_sqrt, lemma=False)

In [ ]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v2_shap, lemma=False)

Evaluate Augmented Shapley Lexicons
F1: 0.151
P 0.181
R 0.184
Skip: 0.009


In [ ]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v3_gptj, lemma=False)

Evaluate Augmented Shapley Lexicons
F1: 0.125
P 0.153
R 0.148
Skip: 0.000


In [19]:
print("Evaluate Augmented Shapley Lexicons")
run_eval(test, lexicon_shapley_augmented_v4_tweetbert, lemma=False)


Evaluate Augmented Shapley Lexicons
F1: 0.146
P 0.177
R 0.180
Skip: 0.009


## Logistic Regression with selected lexicons

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

class Preprocessor:
  def __init__(self, vocabs):
    self.vocabs = vocabs
  
  def run(self, sent):
    words = preprocess(sent, lemma=True)

    # Remove words that are not in vocabs
    words = [w for w in words if w in self.vocabs]
    return words

def dummy_word_tokenize(sent):
  return sent

def train_reg(data, lexicon, **kwargs):
  vocabs = set()
  for label in lexicon:
    vocabs.update(lexicon[label].keys())

  preprocessor = Preprocessor(vocabs)
  vectorizer = CountVectorizer(
      preprocessor=preprocessor.run,
      tokenizer=dummy_word_tokenize
  )

  X_train = vectorizer.fit_transform(data["text"].values)

  y_train = [label_map[l] for l in data[target_column].values]
  
  print("#Vocab", len(vectorizer.get_feature_names_out()))

  model = LogisticRegression(random_state=42, penalty="l2")
  model.fit(X_train, y_train)

  return model, vectorizer


In [21]:
def run_eval_feat(data, model, vectorizer, **kwargs):
  y_test = [label_map[l] for l in data[target_column].values]
  X_test = vectorizer.transform(data["text"].values)

  y_predict = model.predict(X_test)
  print("F1: %.3f" % sklearn.metrics.f1_score(y_test, y_predict, average='macro'))
  print("Acc: %.3f" % sklearn.metrics.accuracy_score(y_test, y_predict))

  p, r, f1, sup = sklearn.metrics.precision_recall_fscore_support(y_test, y_predict, average="macro")
  print("P: %.3f" % p)
  print("R: %.3f" % r)
  return 

In [ ]:
print("Evaluate PMI Lexicons")
model, vectorizer = train_reg(train, lexicon_pmi)
run_eval_feat(test, model, vectorizer)

Evaluate PMI Lexicons
#Vocab 4953
F1: 0.092
Acc: 0.231
P: 0.144
R: 0.108


In [ ]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley)
run_eval_feat(test, model, vectorizer)


Evaluate Shapley Lexicons
#Vocab 2510
F1: 0.260
Acc: 0.374
P: 0.292
R: 0.251


In [ ]:
# print("Evaluate Shapley Lexicons")
# model, vectorizer = train_reg(train, lexicon_shapley_augmented_v1)
# run_eval_feat(test, model, vectorizer)

In [ ]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v2_shap)
run_eval_feat(test, model, vectorizer)

Evaluate Shapley Lexicons
#Vocab 2732
F1: 0.261
Acc: 0.375
P: 0.294
R: 0.252


In [ ]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v3_gptj)
run_eval_feat(test, model, vectorizer)

Evaluate Shapley Lexicons
#Vocab 4464
F1: 0.247
Acc: 0.376
P: 0.288
R: 0.243


In [22]:
print("Evaluate Shapley Lexicons")
model, vectorizer = train_reg(train, lexicon_shapley_augmented_v4_tweetbert)
run_eval_feat(test, model, vectorizer)

Evaluate Shapley Lexicons
#Vocab 2510
F1: 0.260
Acc: 0.374
P: 0.292
R: 0.251
